In [98]:
from __future__ import print_function
import pickle 
import numpy as np
import argparse
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [99]:
mse = torch.nn.MSELoss()

# Train Model

In [100]:
# train_loader = torch.utils.data.DataLoader(trainset_new, batch_size=64, shuffle=True)
# valid_loader = torch.utils.data.DataLoader(validset, batch_size=64, shuffle=True)

In [101]:
trainset_imoprt = pickle.load(open("../data/kaggle/train_labeled.p", "rb"))
validset_import = pickle.load(open("../data/kaggle/validation.p", "rb"))

In [102]:
train_loader = torch.utils.data.DataLoader(trainset_imoprt, batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(validset_import, batch_size=64, shuffle=True)

In [179]:
class Ladder(nn.Module):
    def __init__(self):
        super(Ladder, self).__init__()
        
        self.noise_level = 0.3
        self.layer_sizes = [1000,500,250,250,250]
        self.layer_sizes[0] = 784
        
        self.clean_z = {}
        self.clz_bn = {}
        self.noisy_z = {}
    
        #self.noiser = self.addNoise()
    def addNoise(self, x, mu = 0, std = 1):
        print("size of tensor to add noise to", x.size(1))
        noise = torch.FloatTensor(len(x), x.size(1))
        noiseV = Variable(noise)
        noiseV.resize_as(x)
        noiseV.data.normal_(mu, std)
        x.data.add_(noiseV.data)
        return x
    
    def encode(self, input):
        #self.input = nn.Identity()()
        print("Check point 1", input.size())
        self.clean_z[0] = input.view(-1, self.layer_sizes[0])
        print("Check point 2", self.clean_z[0].size())
        self.noisy_z[0] = self.addNoise(self.clean_z[0])
        print("Check point 3", self.noisy_z[0].size())
        prev_out = self.noisy_z[0]
        print("Check point 4")
        bn_layers = {}
        
        for i in range(1,len(self.layer_sizes)) :
            sz = self.layer_sizes[i]
            self.clean_z[i] = nn.Linear(self.layer_sizes[i-1], sz)(prev_out)
            print("Check point 5")
            bn_layers[i] = nn.BatchNorm1d(sz, 0, 0)
            print("Check point 5.1", self.clean_z[i].size())
            self.clz_bn[i] = bn_layers[i](self.clean_z[i])
            print("Check point 6")
            self.noisy_z[i] = self.addNoise(self.clz_bn[i])
            print("Check point 7", self.noisy_z[i].size(),sz)
            prev_out = F.relu(self.noisy_z[i].add(sz))
            print("Check point 8", prev_out.size())
       
        y = nn.Linear(250, 10)(prev_out)
        y_bn = nn.BatchNorm1d(10)(y)
        print("Output ", y_bn.size())
        #y_softmax = nn.SoftMax()(y_bn)
        #print("output size ", y_softmax.size())
        return y_bn
    
    def forward(self, input):
        output = self.encode(input.view(-1,784))
        return output
model = Ladder()
        

In [180]:
input = Variable(torch.Tensor(64, 1, 28,28))
output = model(input)
print(type(output))

Check point 1 torch.Size([64, 784])
Check point 2 torch.Size([64, 784])
size of tensor to add noise to 784
Check point 3 torch.Size([64, 784])
Check point 4
Check point 5
Check point 5.1 torch.Size([64, 500])
Check point 6
size of tensor to add noise to 500
Check point 7 torch.Size([64, 500]) 500
Check point 8 torch.Size([64, 500])
Check point 5
Check point 5.1 torch.Size([64, 250])
Check point 6
size of tensor to add noise to 250
Check point 7 torch.Size([64, 250]) 250
Check point 8 torch.Size([64, 250])
Check point 5
Check point 5.1 torch.Size([64, 250])
Check point 6
size of tensor to add noise to 250
Check point 7 torch.Size([64, 250]) 250
Check point 8 torch.Size([64, 250])
Check point 5
Check point 5.1 torch.Size([64, 250])
Check point 6
size of tensor to add noise to 250
Check point 7 torch.Size([64, 250]) 250
Check point 8 torch.Size([64, 250])
Output  torch.Size([64, 10])
<class 'torch.autograd.variable.Variable'>


In [539]:
model = torch.load("decon-vae-50.p")
model_name = 'denoise'

optimizer = optim.SGD(model.parameters(), lr = 0.0001)

In [540]:
# CPU only training
def generate_train(epoch):
    model.train()
    train_loss = 0
    classification_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        #
        data, target = Variable(data), Variable(target)
        
        # Add noise
        if model_name == 'denoise':
            noise = torch.FloatTensor(len(data), 1,28,28)
            noiseV = Variable(noise)
            noiseV.resize_as(data)
            noiseV.data.normal_(0, 1)
            data.data.add_(noiseV.data)
        
        optimizer.zero_grad()
        x_new, mu, logvar = model(data)
        kl_loss = 0.5 * torch.mean(torch.exp(logvar) + mu ** 2 - 1. - logvar)
        recon_loss = mse(x_new, data) + kl_loss
        output = F.log_softmax(mu)
        class_loss = F.nll_loss(output, target)
        loss = recon_loss + class_loss
        loss.backward()

        train_loss += loss.data[0]
        classification_loss += class_loss.data[0]
        optimizer.step()
    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, classification_loss / len(train_loader.dataset)))

    
def generate_test(epoch, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        x_new, mu, logvar = model(data)
        output = F.log_softmax(mu) 
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()
        

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))




In [ ]:
# Generative Classifier
for epoch in range(1, 50):
    generate_train(epoch)
    generate_test(epoch, valid_loader)

In [537]:
# Save model
torch.save(model, "decon-vae-50.p")


In [247]:
# Check generated images
from  torchvision.utils import save_image
for index, (data, target) in enumerate(train_loader):
    data, target = Variable(data), Variable(target)
    gen_data, mu, logvar = model(data)
    gen_images = gen_data.view(-1, 1,28,28)
    #print(data[0])
    save_image(gen_images.data, 'images/gen{}.jpg'.format(index))


In [ ]:
# CPU only training
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.8)
def train(epoch):
    classifier.train()
    for batch_idx, (data, target) in enumerate(train_loader):

        data, target = Variable(data), Variable(target)
        gen_data, mu, logvar = model(data)
        #Check reconstruction loss 
        recon_loss = reconstruction_function(gen_data, data)
        #print("Reconstruction loss is", recon_loss)
        #print("Type of generated data", type(gen_data))
        
        optimizer.zero_grad()
        output = classifier(gen_data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test(epoch, valid_loader):
    classifier.eval()
    test_loss = 0
    correct = 0
    for data, target in valid_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        gen_data, mu, logvar = model(data)
        output = classifier(gen_data)
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(valid_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(valid_loader.dataset),
        100. * correct / len(valid_loader.dataset)))
    


In [ ]:
#Classification phase
for epoch in range(1, 10):
    train(epoch)
    test(epoch, valid_loader)

# Create Sample Submission

In [ ]:
testset = pickle.load(open("../data/kaggle/test.p", "rb"))

In [ ]:
test_loader = torch.utils.data.DataLoader(testset,batch_size=64, shuffle=False)

## Test Accuuracy

In [ ]:
test(1, test_loader)

In [ ]:
label_predict = np.array([])
model.eval()
for data, target in test_loader:
    data, target = Variable(data, volatile=True), Variable(target)
    output = model(data)
    temp = output.data.max(1)[1].numpy().reshape(-1)
    label_predict = np.concatenate((label_predict, temp))

In [ ]:
label_predict

In [ ]:
predict_label = pd.DataFrame(label_predict, columns=['label'], dtype=int)
predict_label.reset_index(inplace=True)
predict_label.rename(columns={'index': 'ID'}, inplace=True)

In [ ]:
predict_label.head()

In [ ]:
predict_label.to_csv('../data/kaggle/sample_submission.csv', index=False)
